In [1]:
# -*- coding: utf-8 -*-
# Normal Python packages
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import numpy as np
from glob import glob
import os

# Plotting packages
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from shapely.geometry import box, LineString, Point
import cmocean as cm

# Add the path to the Tatsu's swot library
import sys
sys.path.append('../src/')
import swot_utils
import data_loaders
import download_swaths


# Turn off warnings
import warnings
warnings.filterwarnings("ignore")


# Load a bathymetry dataset for plotting (optional)

In [2]:
def load_bathymetry(zip_file_url):
    """Read zip file from Natural Earth containing bathymetry shapefiles"""
    # Download and extract shapefiles
    import io
    import zipfile
    import os

    import requests
    
    # Download bathymetry if you don't have it already..
    if not os.path.exists("../ne_10m_bathymetry_all/"):
        import requests
        r = requests.get(zip_file_url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall("../ne_10m_bathymetry_all/")
    
    # Read shapefiles, sorted by depth
    shp_dict = {}
    files = glob('../ne_10m_bathymetry_all/*.shp')
    assert len(files) > 0
    files.sort()
    depths = []
    for f in files:
        depth = '-' + f.split('_')[-1].split('.')[0]  # depth from file name
        depths.append(depth)
        bbox = (-180, -90, 180, 90)  # (x0, y0, x1, y1)
        nei = shpreader.Reader(f, bbox=bbox)
        shp_dict[depth] = nei
    depths = np.array(depths)[::-1]  # sort from surface to bottom
    return depths, shp_dict


if __name__ == "__main__":
    # Load data (14.8 MB file)
    depths_str, shp_dict = load_bathymetry(
        'https://naturalearth.s3.amazonaws.com/' +
        '10m_physical/ne_10m_bathymetry_all.zip')

    # Construct a discrete colormap with colors corresponding to each depth
    depths = depths_str.astype(int)
    N = len(depths)
    nudge = 0.01  # shift bin edge slightly to include data
    boundaries = [min(depths)] + sorted(depths+nudge)  # low to high
    norm = matplotlib.colors.BoundaryNorm(boundaries, N)
    blues_cm = matplotlib.colormaps['Blues_r'].resampled(N)
    colors_depths = blues_cm(norm(depths))

# A notebook to try some plotting

# Load the swath/pass combos you want

Assuming you already have everything downloaded..

In [3]:
# Specify the path to the subsetted data
L3_cali_path = f"../SWOT_L3/Unsmoothed_cali"

# Define domain
# Rough West of California domain
cali_sw_corner = [-125,38.5] # full area [-134.0994348261152,27.77869536057762]
cali_ne_corner = [-123,42.5] # full area [-115.2951365210726,45.43712363060855]
lat_lims = [cali_sw_corner[1],cali_ne_corner[1]]

# Define mission phase (1-day repeat vs science) and 
# cycles we are interested in
# Use sph_calval_swath for the 1-day repeats
path_to_sph_file="../orbit_data/sph_calval_swath.zip"
# Cycles 474 - 578 are from the 1-day repeat 
cycles = [str(c_num).zfill(3) for c_num in range(474,579)] # 579

pass_IDs_list = download_swaths.find_swaths(cali_sw_corner, cali_ne_corner,
                                           path_to_sph_file=path_to_sph_file)

# Be sure to set subset as True when taking a subset of the data defined in notebook 2!

cycle_data = {}
for cycle in cycles:
    cycle_data[cycle] = data_loaders.load_cycle(L3_cali_path,fields=["time","ssha","ssha_unedited","ssha_noiseless","sigma0"],
                                                cycle=cycle,pass_ids=pass_IDs_list,subset=True,lats=lat_lims
                                               )


Loading SWOT_L3_LR_SSH_Unsmoothed_474_013_20230329T081622_20230329T090619_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_478_013_20230402T073744_20230402T082850_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_479_013_20230403T072822_20230403T081927_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_480_013_20230404T071859_20230404T081005_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_481_013_20230405T070937_20230405T080043_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_482_013_20230406T070015_20230406T075121_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_483_013_20230407T065053_20230407T074159_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_484_013_20230408T064131_20230408T073237_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_485_013_20230409T063210_20230409T072315_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_486_013_20230410T062248_20230410T071353_v1.0.2_cali.nc
Loading SWOT_L3_LR_SSH_Unsmoothed_487_013_20230411T061326_20230411T070432_v1.0.2_cali.nc
Loading SWOT_L3_LR_SS

In [ ]:
cycles

## Print the start and end times of the swaths to plot
Note this is the mean time over the section you are plotting

In [ ]:
swaths_to_plot_subset = cycle_data["474"][0]
swaths_to_plot_subset

In [ ]:
swaths_to_plot_subset.where(num_pixels=slice(40, 42.5))

In [ ]:
print(swaths_to_plot_subset[0].time.mean().values)
print(swaths_to_plot_subset[-1].time.mean().values)


# Example Plotting Script


For the full swath, we want to have the following parameters:

target_width = 3244

target_height = 5969

figsize = (target_width / dpi, target_height / dpi)

fixed_extent = [-129.641259, -121.15456099999994, 26.775556, 46.482364000000004]

gl.xlabel_style (and for y): 'size' = 25

axs.set_title(title, fontsize=100, pad=40) # fontsize = 100

cbar.set_label(cbar_title, rotation=270, fontsize=80, labelpad=40)

cbar.ax.tick_params(labelsize=50)  # Increase the font size of colorbar ticks

In [4]:
def plot_cycle(swaths, field="ssha", title="Example Swaths August 2024 (Cycle 4)", cbar_title="SSHA (m)",
              subplot_kw={'projection': ccrs.PlateCarree()},dpi=300,
              ssha_plot_kw={"cmap": cm.cm.balance, "transform": ccrs.PlateCarree(),
                            "s": 1, "vmin": -0.3, "vmax": 0.3, "marker": ".", "alpha": 1, "linewidths": 0},
              fixed_extent=[-125, -123, 38.5, 42.5]):
    
    # [-129.641259, -121.15456099999994, 26.775556, 46.482364000000004] for final gif 

    """
    Plots satellite swath data on a map using Cartopy and Matplotlib.

    Args:
        swaths (list): A list of swath xarray dataset objects containing latitude, longitude, time, cycle, and variable data.
        field (str, optional): Name of the dataarray to be visualized (default is "ssha").
        title (str, optional): The title of the plot (default is "Example Swaths August 2024 (Cycle 4)").
        cbar_title (str, optional): The title of the colorbar (default is "SSHA (m)").
        subplot_kw (dict, optional): Keyword arguments for the subplot, including projection settings.
        dpi (int): Dots per inch, i.e. the resolution of the plot.
        ssha_plot_kw (dict, optional): Keyword arguments for the scatter plot, including color map, markers, and range settings.
    """
    
    # Add labels to keep track of specific swaths
    swath_labels = []
    for swath in swaths:
        label_lat = np.around(swath.latitude[-1, :].min().values, 2)
        label_lon = np.around(swath.longitude[-1, :].min().values, 2)
        label_cycle = swath.cycle
        label_pass = swath.pass_ID
        label_time = swath.time.mean().values.astype(str)[:19]
        swath_labels.append({"label_lat": label_lat,
                             "label_lon": label_lon,
                             "label_cycle": label_cycle,
                             "label_pass": label_pass,
                             "label_time": label_time
                             })

    # Initialize plot


    # Define target dimensions in pixels
    target_width = 5000 #3244
    target_height = 10000 #5969
    
    # Set figure size based on target dimensions and desired DPI
    fig, axs = plt.subplots(1, 1, figsize = (10, 20), subplot_kw=subplot_kw, dpi=dpi)

    # figsize = (target_width / dpi, target_height / dpi)

    # Add some geographic features (coastline and land mask)
    axs.add_feature(cfeature.COASTLINE.with_scale('10m'))
    axs.add_feature(cfeature.LAND, edgecolor='none', facecolor='lightgray')

    # Set a fixed geographic extent
    axs.set_extent(fixed_extent, crs=ccrs.PlateCarree())

    # Manually set axis limits to lock the map boundaries
    axs.set_xlim(fixed_extent[0], fixed_extent[1])
    axs.set_ylim(fixed_extent[2], fixed_extent[3])
    
    # Add bathymetry features based on depth levels
    for i, depth_str in enumerate(depths_str):
        axs.add_geometries(shp_dict[depth_str].geometries(),
                           crs=ccrs.PlateCarree(),
                           color=colors_depths[i])
    
    # Plot the first swath to set up the colorbar
    cax = axs.scatter(swaths[0].longitude[:, :], swaths[0].latitude[:, :],
                      c=swaths[0][field][:, :], **ssha_plot_kw, zorder=10)
    
    # Add colorbar and gridlines
    cbar = plt.colorbar(cax, ax=axs, shrink=0.4, pad=0.12)
    gl = axs.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                       linewidth=1, color='gray', alpha=0.5, linestyle='--')
    
    # Directly change the font size for tick labels
    gl.xlabels_top = False  # disable top labels
    gl.ylabels_right = False  # disable right labels
    gl.xlabel_style = {'size': 15, 'color': 'black'}  # x-axis label size
    gl.ylabel_style = {'size': 15, 'color': 'black'}  # y-axis label size
    
    # Overlay remaining swaths with incremental z-order
    for i, plt_swath in enumerate(swaths[1:]):
        print(i, end=",")  # Debugging output to track progress
        axs.scatter(plt_swath.longitude[:, :], plt_swath.latitude[:, :],
                    c=plt_swath[field][:, :], **ssha_plot_kw, zorder=10 + i)
    
    # Adjust colorbar labels if plotting quality flags
    if field == "quality_flag":
        ticks = np.arange(0, 10, 1)
        cbar.ax.set_yticks(ticks)
        cbar.ax.set_yticklabels(swaths[0].quality_flag.flag_meanings.split(" "))
    
    # Add textual labels to swaths
    for swath_label in swath_labels:
        txt = axs.text(swath_label["label_lon"] - 360, swath_label["label_lat"] - 0.5,
                       (f"Cycle {swath_label["label_cycle"]} \n"
                        + f"Pass #{swath_label["label_pass"]} \n"
                        + f"{swath_label["label_time"][:10]} \n"
                        + f"{swath_label["label_time"][10:19]} " ),
                       fontsize=20, weight='bold', zorder=len(swaths) + 30)
        txt.set_bbox(dict(facecolor='white', alpha=1, edgecolor='k'))
    
    # Set axis labels and title
    axs.set_ylabel("Latitude (deg)", fontsize=100)
    axs.set_xlabel("Longitude (deg)", fontsize=100)
    axs.set_title(title, fontsize=30, pad=40) # fontsize = 100
    cbar.set_label(cbar_title, rotation=270, fontsize=30, labelpad=40)
    cbar.ax.tick_params(labelsize=15)  # Increase the font size of colorbar ticks

    # Change font size of gridline labels (latitude and longitude coordinates)
    # gl.xlabels_fontsize = 50  # Latitude ticks font size
    # gl.ylabels_fontsize = 50  # Longitude ticks font size

    current_extent = axs.get_extent(crs=ccrs.PlateCarree())
    print("Current map extent:", current_extent)    

    # Optimize layout and display the plot
    # fig.tight_layout()
    fig.savefig(f"../cali_swaths_unsmoothed_subset_3/{swath_label["label_cycle"]}.png", dpi=dpi) #, bbox_inches="tight")
    # plt.show()
    # plt.close()


# Histogram Plot

In [ ]:
ssha_swath_data = []

def data_flattener(swaths, field="ssha"):
    for i, plt_swath in enumerate(swaths[0:]):
        # print(i, end=",")  # Debugging output to track progress
        ssha_swath_data.append(plt_swath[field].values.flatten())
        plt.hist(plt_swath[field].values.flatten(), density=True, edgecolor='black', bins=300)
        plt.title(f"Distribution of SSHA for California Swaths \n  (Cycle {cycle}, {cycle_data[cycle][0].time_coverage_begin[:10]} to {cycle_data[cycle][-1].time_coverage_end[:10]})")
        plt.xlim(-0.3,0.3)

for cycle in cycles:
    print(cycle)
    
    if len(cycle_data[cycle])<1:
        print(f"No swaths loaded for cycle {cycle}! Skipping...")
        continue

    else:
        data_flattener(cycle_data[cycle])

# title=f"Distribution of SSHA for California Swaths \n  (Cycle {cycle}, {cycle_data[cycle][0].time_coverage_begin[:10]} to {cycle_data[cycle][-1].time_coverage_end[:10]})"

#test1 = cycle_data['474'][1].ssha.values.flatten() #.latitude[:, :].values
#test2 = cycle_data['477'][0].ssha.values.flatten() #.latitude[:, :].values

ssha_swath_data

#plt.hist(test1, density=True, color='blue', edgecolor='black', bins=100)

#for arr in ssha_swath_data:
#    plt.hist(arr, density=True, edgecolor='black', bins=100)
#    plt.xlim(-0.3,0.3)

In [ ]:
#test1 = cycle_data['474'][1]['ssha'] #.flatten() #.latitude[:, :].values
#test1.values.flatten()
#test1.values.flatten()

# Try to plot everything

I'm plotting all of the swaths we managed to pull for each cycle. NOTE: Some passes in each cycle might be missing! If that is the case the specific pass may have either failed to download or simply not be available (some of the earlier cycles during the fast repeat phase are missing a large number of passes due to instrument calibration). For a quick check of whether the pass is available on AVISO just sftp into AVISO and navigate to the cycle folder for the data release, for example for the V1.0.2 L3 data release on the server, <code>ls /swot_products/l3_karin_nadir/l3_lr_ssh/v1_0_2/Unsmoothed/cycle_477</code> should be missing both passes "004" and "019"

# GCM filters
# sci-py convolutional/gaussian filters

Making movies down here. Generally, on my machine which has 64 GB RAM, we want to keep image files under ~25 MB individually to not run out of memory.

In [ ]:
cycles = [str(c_num).zfill(3) for c_num in range(474,579)]


for cycle in cycles:
    print(cycle)
    
    if len(cycle_data[cycle])<1:
        print(f"No swaths loaded for cycle {cycle}! Skipping...")
        continue

    else:
        plot_cycle(cycle_data[cycle],title=f"Example California Swaths \n  (Cycle {cycle}, {cycle_data[cycle][0].time_coverage_begin[:10]} to {cycle_data[cycle][-1].time_coverage_end[:10]})")


474
Current map extent: (-125.0, -123.0, 38.5, 42.5)
475
No swaths loaded for cycle 475! Skipping...
476
No swaths loaded for cycle 476! Skipping...
477
No swaths loaded for cycle 477! Skipping...
478
Current map extent: (-125.0, -123.0, 38.5, 42.5)
479
Current map extent: (-125.0, -123.0, 38.5, 42.5)
480
Current map extent: (-125.0, -123.0, 38.5, 42.5)
481
Current map extent: (-125.0, -123.0, 38.5, 42.5)
482
Current map extent: (-125.0, -123.0, 38.5, 42.5)
483
Current map extent: (-125.0, -123.0, 38.5, 42.5)
484
Current map extent: (-125.0, -123.0, 38.5, 42.5)
485
Current map extent: (-125.0, -123.0, 38.5, 42.5)


KeyboardInterrupt: 

# Making movies

In [1]:
import imageio as iio
import glob

def create_gif_from_images(image_folder="../cali_swaths_unsmoothed", 
                           output_path="../movies/swaths_animation.gif", 
                           fps=60, 
                           image_format="*.png"):
    """
    Creates a GIF from existing images in a specified folder.

    Args:
        image_folder (str, optional): Folder containing the images (default is "../cali_swaths_unsmoothed").
        output_path (str, optional): Path to save the output GIF (default is "swaths_animation.gif").
        fps (int, optional): Frames per second for the GIF animation (default is 2).
        image_format (str, optional): Format of the images to include (default is "*.png").
    """
    # Collect image file paths and sort them by filename
    image_files = sorted(glob.glob(f"{image_folder}/{image_format}"))
    
    if not image_files:
        print("No images found in the specified folder.")
        return
    
    # Read the images into a list
    images = [iio.imread(image_file) for image_file in image_files]
    
    # Write the images as a GIF
    iio.mimwrite(output_path, images, duration=1/fps, loop = 0)

    print(f"GIF saved to {output_path}")


In [2]:
create_gif_from_images(image_folder="../cali_swaths_unsmoothed_subset_2", 
                       output_path="../movies/swaths_animation_subset.gif", 
                       fps=180)

C:\Users\david\AppData\Local\Temp\ipykernel_39096\502416517.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [iio.imread(image_file) for image_file in image_files]


GIF saved to ../movies/swaths_animation_subset.gif
